In [18]:
%matplotlib inline
import numpy as np
import os
import os.path as osp
import gym
import imageio
import matplotlib.pyplot as plt

from torch import nn
from IPython.display import Image, Video
from gym import wrappers
from spinup.utils import rrc_utils, test_policy
from rrc_simulation.gym_wrapper.envs import cube_env

In [2]:
env_fn = rrc_utils.make_env_fn('rrc_simulation.gym_wrapper:real_robot_challenge_phase_push-v1',
                               enable_cameras=True)
env = env_fn()

In [ ]:
env.reset()
plt.imshow(env.render())

In [4]:
env_fn = rrc_utils.rrc_ppo_env_fn
ac_kwargs = dict(hidden_sizes=[64,64], activation=nn.Tanh)
logger_kwargs = dict(output_dir='./data/ppo_rrc_cube', exp_name='cube_rrc')

In [39]:
load_path = '/scr1/Developer/Projects/spinningup/data/ppo_rrc_pusht1/ppo_rrc_pusht1_s0'
exp_name = osp.split(load_path)[1]

vid_dir = osp.join('./videos', exp_name)
if not osp.exists(vid_dir):
    os.makedirs(vid_dir)

i = 0
vid_save_path = lambda: osp.join(vid_dir, 'ep_{}.mp4'.format(i))

while osp.exists(vid_save_path()): i +=1 
vid_save_path = vid_save_path()

In [41]:
env, policy = test_policy.load_policy_and_env(load_path, 'last', True)

if env is None:
    env = rrc_utils.push_ppo_env_fn()
    env.unwrapped.enable_cameras = True



Loading from /scr1/Developer/Projects/spinningup/data/ppo_rrc_pusht1/ppo_rrc_pusht1_s0/pyt_save/model.pt.




/scr1/.pyenv/versions/miniconda3-latest/envs/rrc/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [46]:
# obs = env.reset()
d = False
frames = [env.render()]

while not d:
    obs, r, d, i = env.step(policy(obs))
    frames.append(env.render())

with imageio.get_writer(vid_save_path) as writer:
    for frame in frames:
        writer.append_data(frame)

ValueError: Given time index t has to match with index of the current step or the next one.

In [44]:
len(frames)

12

In [22]:
obs

{'robot_position': array([ 0.        ,  1.22173047, -2.26892805,  0.        ,  1.22173047,
        -2.26892805,  0.        ,  1.22173047, -2.26892805]),
 'robot_velocity': array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'robot_tip_positions': array([[ 0.08457   ,  0.06003509,  0.15397678],
        [ 0.00970691, -0.10325731,  0.15397678],
        [-0.09427691,  0.04322223,  0.15397678]]),
 'object_position': array([ 0.01153694, -0.06141343,  0.0325    ]),
 'object_orientation': array([ 0.        ,  0.        ,  0.93348761, -0.35860964]),
 'goal_object_position': array([-0.00781044,  0.1087707 ,  0.0325    ])}

In [ ]:
ppo(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=5000, epochs=250, logger_kwargs=logger_kwargs)